# Classifying Images with CNNs

In this exercise you will design a Convolutional Neural Network (CNN) for MNIST. CNNs are the workhorses of modern computer vision.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

from keras.datasets import mnist
from keras.models import Sequential
from keras.optimizers import SGD, RMSprop
from keras.utils import np_utils

Using TensorFlow backend.


## CNN

Let's build a convolutional model! For this, we need to have the data in its original shape. Also note that when we reshape the data below, we add a dimension of 1 - this is the number of **channels** in the image, which is just 1 because these are grayscale images. If they were color, this would be 3 for RGB. Make sure you understand how and why we are preparing the data below.

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print X_train.shape

y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

(60000, 28, 28, 1)


### Exercise 1 - design a CNN for MNIST

It is up to you what the model will be. Here are some things you need to decide:
* how many convolutional layers?
* what spatial size will your convolutions be?
* how many channels will your convolutions be?
* what nonlinearity will you use?
* will you use pooling? what type?
* how many fully-connected layers will you have?
* will you use dropout?
* what batch size?

Keras provides a special layer called `Flatten` to flatten the convolutional features into a vector before the fully-connected layers. You should look at the documentation for Keras's convolutional layers: http://keras.io/layers/convolutional/. In particular, you may want to look at `Convolution2D`, `MaxPooling2D`, `AveragePooling2D`, `Flatten`, and `Dropout`. For this problem, you make want to use the `'rmsprop'` optimizer - it is an algorithm that adapts the learning rate during learning for you automatically.

Can you get to 98% accuracy? You shouldn't need more than a few epochs to do pretty well.

Suggestions:
* Try using at least 2 convolutional layers. This should get you off to a good start and it will come in handy later.

In [ ]:
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D, Flatten
from keras.optimizers import Adam

model = Sequential()

# Design a CNN

model.add(Convolution2D(32, 3, 3,
                        border_mode='valid',
                        input_shape=(28,28,1)))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

opt = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# Fit your model
model.fit(X_train, y_train, nb_epoch=20, batch_size=16, verbose=2, validation_split=0.8)

Train on 11999 samples, validate on 48001 samples
Epoch 1/20
68s - loss: 0.4696 - acc: 0.8590 - val_loss: 0.1424 - val_acc: 0.9549
Epoch 2/20
64s - loss: 0.2935 - acc: 0.9176 - val_loss: 0.1548 - val_acc: 0.9529
Epoch 3/20
76s - loss: 0.2553 - acc: 0.9244 - val_loss: 0.1363 - val_acc: 0.9619
Epoch 4/20
79s - loss: 0.2422 - acc: 0.9292 - val_loss: 0.1196 - val_acc: 0.9648
Epoch 5/20
64s - loss: 0.2284 - acc: 0.9351 - val_loss: 0.1528 - val_acc: 0.9586
Epoch 6/20
64s - loss: 0.2148 - acc: 0.9364 - val_loss: 0.1298 - val_acc: 0.9622
Epoch 7/20
64s - loss: 0.2150 - acc: 0.9365 - val_loss: 0.1282 - val_acc: 0.9636
Epoch 8/20
62s - loss: 0.2050 - acc: 0.9407 - val_loss: 0.1345 - val_acc: 0.9615
Epoch 9/20
65s - loss: 0.2195 - acc: 0.9387 - val_loss: 0.1308 - val_acc: 0.9667
Epoch 10/20
71s - loss: 0.2141 - acc: 0.9416 - val_loss: 0.1351 - val_acc: 0.9621
Epoch 11/20
68s - loss: 0.1996 - acc: 0.9459 - val_loss: 0.1205 - val_acc: 0.9699
Epoch 12/20


In [7]:
score = model.evaluate(X_test, y_test, verbose=2)
print('Test score:', score[0])
print('Test accuracy:', score[1])

('Test score:', 0.1123644335470628)
('Test accuracy:', 0.96789999999999998)


## Saving a trained model

Keras allows you to save and restore models. The model structure can be saved as JSON and the parameters as HDF5. Save your trained model for later:

In [ ]:
with open('mnist_cnn.json', 'w') as f:
    f.write(model.to_json())

model.save_weights('mnist_cnn_weights.h5')